In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.datasets import fetch_openml
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [15]:
# Load Boston Housing dataset and convert to pandas DataFrame
boston = fetch_openml(
    name='boston', version=1, as_frame=True
)
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['MEDV'] = boston.target  # Add target column to the DataFrame

In [16]:
df.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   CRIM     506 non-null    float64 
 1   ZN       506 non-null    float64 
 2   INDUS    506 non-null    float64 
 3   CHAS     506 non-null    category
 4   NOX      506 non-null    float64 
 5   RM       506 non-null    float64 
 6   AGE      506 non-null    float64 
 7   DIS      506 non-null    float64 
 8   RAD      506 non-null    category
 9   TAX      506 non-null    float64 
 10  PTRATIO  506 non-null    float64 
 11  B        506 non-null    float64 
 12  LSTAT    506 non-null    float64 
 13  MEDV     506 non-null    float64 
dtypes: category(2), float64(12)
memory usage: 49.0 KB


In [18]:
# Features and target
X = df.drop(columns=['MEDV'])
y = df['MEDV']

In [19]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=27
)

In [20]:
# Scaling
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
# ElasticNet model
elastic_net = ElasticNet(random_state=27)

# Hyperparameter grid
alphas = np.logspace(-3, 2, 10)
l1_ratios = np.round(np.linspace(0.1, 1.0, 10), 4)

param_grid = {
    'alpha': alphas
    , 'l1_ratio': l1_ratios
}

# GridSearchCV setup
grid_search = GridSearchCV(
    estimator=elastic_net
    , param_grid=param_grid
    , scoring='neg_mean_squared_error'
    , cv=5
)

In [22]:
# Fit grid search
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ElasticNet(random_state=27),
             param_grid={'alpha': array([1.00000000e-03, 3.59381366e-03, 1.29154967e-02, 4.64158883e-02,
       1.66810054e-01, 5.99484250e-01, 2.15443469e+00, 7.74263683e+00,
       2.78255940e+01, 1.00000000e+02]),
                         'l1_ratio': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='neg_mean_squared_error')

In [23]:
grid_search.best_params_

{'alpha': 0.046415888336127795, 'l1_ratio': 0.1}

In [24]:
# Best model
best_model = grid_search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test)

In [25]:
# Metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
rmse_ratio = rmse / np.mean(y_test)

metrics_summary = {
    'MSE': mse
    , 'RMSE': rmse
    , 'R² Score': r2
    , 'RMSE Ratio': rmse_ratio
}
metrics_summary

{'MSE': 27.765887564737078,
 'RMSE': 5.269334641559319,
 'R² Score': 0.693540649579601,
 'RMSE Ratio': 0.23390727367005423}

In [26]:
# Convert grid results to DataFrame
results_df = pd.DataFrame(grid_search.cv_results_)

# Pivot table for heatmap
heatmap_data = results_df.pivot_table(
    index='param_l1_ratio'
    , columns='param_alpha'
    , values='mean_test_score'
)

# Convert scores from negative MSE to positive
heatmap_data = -1 * heatmap_data

# Plotly Express heatmap
fig = px.imshow(
    heatmap_data, width=1000, height=800
    , labels={"color": "MSE"}
    , x=heatmap_data.columns.astype(str)  # Convert alpha values to strings for cleaner display
    , y=heatmap_data.index.astype(str)   # Convert l1_ratio values to strings for cleaner display
    , color_continuous_scale="YlGnBu"
    , text_auto=".2f"
)

# Add title and labels
fig.update_layout(
    title="Grid Search CV Results (MSE)"
    , xaxis_title="Alpha"
    , yaxis_title="L1 Ratio"
    , font=dict(size=14)
    , xaxis=dict(tickangle=-45)  # Rotate x-axis labels to avoid overlap
    , yaxis=dict(tickangle=-90)
)

# Show the figure
fig.show()